In [30]:
import graphlab

In [31]:
people = graphlab.SFrame('people_wiki.gl/')

In [32]:
people.num_rows()

59071

In [33]:
people['word_count'] = graphlab.text_analytics.count_words(people['text'])

In [34]:
#compute tfidf SArray
tfidf = graphlab.text_analytics.tf_idf(people['word_count']) 
# add as a columnt to people SF
people['tfidf'] = tfidf

### count words for john

In [35]:
elton = people[people['name'] == 'Elton John']

In [57]:
elton

Columns:
	URI	str
	name	str
	text	str
	word_count	dict
	tfidf	dict

Rows: 1

Data:
+-------------------------------+------------+-------------------------------+
|              URI              |    name    |              text             |
+-------------------------------+------------+-------------------------------+
| <http://dbpedia.org/resour... | Elton John | sir elton hercules john cb... |
+-------------------------------+------------+-------------------------------+
+-------------------------------+-------------------------------+
|           word_count          |             tfidf             |
+-------------------------------+-------------------------------+
| {'all': 1L, 'six': 1L, 'pr... | {'all': 1.6431112434912472... |
+-------------------------------+-------------------------------+
[1 rows x 5 columns]

In [45]:
#for ref
obama = people[people['name'] == 'Barack Obama']

elton['word_count'] = graphlab.text_analytics.count_words(elton['text'])

In [36]:
## tranform dictionary into table and stort it (chain call)
elton_wcnt_table = elton[['word_count']].stack('word_count', new_column_name = ['word','count']).sort('count',ascending=False)

In [37]:
# quiz words most frequent
elton_wcnt_table.head(3)

word,count
the,27
in,18
and,15


In [38]:
# quiz words less frequent
elton_wcnt_table.sort('count',ascending=True).head(3)

word,count
1994,1
samesex,1
solo,1


In [43]:
elton['tfidf']

dtype: dict
Rows: 1
[{'all': 1.6431112434912472, 'six': 2.4538226269605703, 'producer': 2.6157162162644934, 'heavily': 4.584578192498009, 'over': 2.975646311911467, 'named': 2.0300155412252816, 'fifty': 5.11719847609142, 'four': 1.922106072733316, 'openly': 5.056906245835299, 'including': 1.2272824458461182, 'highestprofile': 8.789270811888976, 'years': 1.0752380994247055, 'its': 3.3751896805390627, 'impact': 4.04144432549936, 'westminster': 5.272762583715825, '27': 3.110616229728885, '21': 5.594501726978586, 'wed': 6.908957945319475, 'had': 1.177428412308558, '1947': 3.756656611073944, 'abbey': 5.693190564500702, 'winning': 2.416330313042851, 'late': 2.374355720500003, 'to': 0.1877797507271969, 'commander': 4.099963746694684, 'about': 1.9265128988375737, 'born': 0.268196273764765, '2014': 2.2073995783446634, 'as': 0.2543390440248236, 'has': 3.8564778576963508, '2013': 1.9545642372230505, 'his': 1.0504458208546292, 'march': 1.9573172463231197, 'than': 6.195113430831934, 'song': 2.86918

### tf-idf quiz

In [48]:
#examine tfidf for elton most high tf-idf quiz
elton[['tfidf']].stack('tfidf',new_column_name=['word','tfidf']).sort('tfidf',ascending=False)

word,tfidf
furnish,18.38947184
elton,17.48232027
billboard,17.3036809575
john,13.9393127924
songwriters,11.250406447
overallelton,10.9864953892
tonightcandle,10.9864953892
19702000,10.2933482087
fivedecade,10.2933482087
aids,10.262846934


In [46]:
#for ref
obama[['tfidf']].stack('tfidf',new_column_name=['word','tfidf']).sort('tfidf',ascending=False)

word,tfidf
obama,43.2956530721
act,27.678222623
iraq,17.747378588
control,14.8870608452
law,14.7229357618
ordered,14.5333739509
military,13.1159327785
involvement,12.7843852412
response,12.7843852412
democratic,12.4106886973


## cosine distances quiz

In [51]:
vicbeck = people[people['name'] == 'Victoria Beckham']
paulm = people[people['name'] == 'Paul McCartney']

In [53]:
graphlab.distances.cosine(elton['tfidf'][0],vicbeck['tfidf'][0])

0.9567006376655429

In [55]:
#elton is closer to Paul McCartney
graphlab.distances.cosine(elton['tfidf'][0],paulm['tfidf'][0])

0.8250310029221779

### k-means quiz
##### 2 k-nn one on feature tfidf , 2nd  on word_count

In [59]:
knn_model = graphlab.nearest_neighbors.create(people,features=['tfidf'],label='name',distance='cosine')

Starting brute force nearest neighbors model training.

In [60]:
knn_wc_model= graphlab.nearest_neighbors.create(people,features=['word_count'],label='name',distance='cosine')

Starting brute force nearest neighbors model training.

In [61]:
#quiz word count Cliff Richard  
knn_wc_model.query(elton)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 13.493ms     |

| Done         |         | 100         | 307.782ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Elton John,2.22044604925e-16,1
0,Cliff Richard,0.16142415259,2
0,Sandro Petrone,0.16822542751,3
0,Rod Stewart,0.168327165587,4
0,Malachi O'Doherty,0.177315545979,5


In [62]:
#quiz tfidf Rod Stewart
knn_model.query(elton)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 7.52ms       |

| Done         |         | 100         | 308.32ms     |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Elton John,-2.22044604925e-16,1
0,Rod Stewart,0.717219667893,2
0,George Michael,0.747600998969,3
0,Sting (musician),0.747671954431,4
0,Phil Collins,0.75119324879,5


In [63]:
#quiz word count Mary Fitzgerald (artist)
knn_wc_model.query(vicbeck)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 10.528ms     |

| Done         |         | 100         | 574.542ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Victoria Beckham,-2.22044604925e-16,1
0,Mary Fitzgerald (artist),0.207307036115,2
0,Adrienne Corri,0.214509782788,3
0,Beverly Jane Fry,0.217466468741,4
0,Raman Mundair,0.217695474992,5


In [64]:
#quiz tfidf David Beckham
knn_model.query(vicbeck)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 7.017ms      |

| Done         |         | 100         | 313.373ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Victoria Beckham,1.11022302463e-16,1
0,David Beckham,0.548169610263,2
0,Stephen Dow Beckham,0.784986706828,3
0,Mel B,0.809585523409,4
0,Caroline Rush,0.819826422919,5


In [71]:
# for personal interest
putin = people[people['name'] == 'Vladimir Putin']
turing  = people[people['name'] == 'Alan Turing']

In [74]:
knn_model.query(putin)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 6.506ms      |

| Done         |         | 100         | 252.161ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Vladimir Putin,0.0,1
0,Igor Putin,0.565020605715,2
0,Vera Putina,0.659680977993,3
0,Dmitry Medvedev,0.687347933213,4
0,Alexei Navalny,0.710051236145,5
